<a href="https://colab.research.google.com/github/Syedrockzz/Titanic-survival-prediction/blob/main/Syed_titanic_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Suppress warnings
from warnings import filterwarnings
filterwarnings(action='ignore')

In [ ]:
# Load the dataset
df = pd.read_csv("/content/Titanic-Dataset.csv")

# Feature Engineering
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
title_age_median = df.groupby('Title')['Age'].transform('median')
df['Age'].fillna(title_age_median, inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Fare'].fillna(df['Fare'].median(), inplace=True)

# Drop unnecessary columns
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Embarked', 'Title'], drop_first=True)

In [ ]:
# Splitting the dataset
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Filling missing ages using Random Forest Regressor
age_df = df[['Age', 'Survived', 'Fare', 'Parch', 'SibSp', 'Pclass']]
age_df_notnull = age_df.loc[df['Age'].notnull()]
age_df_isnull = age_df.loc[df['Age'].isnull()]
X_train_age = age_df_notnull.drop('Age', axis=1)
y_train_age = age_df_notnull['Age']
X_test_age = age_df_isnull.drop('Age', axis=1)

RFR = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
RFR.fit(X_train_age, y_train_age)
if not X_test_age.empty:
    predicted_ages = RFR.predict(X_test_age)
    df.loc[df['Age'].isnull(), 'Age'] = predicted_ages

In [ ]:
# Train Random Forest Classifier
RFC = RandomForestClassifier(n_estimators=1000, random_state=42)
RFC.fit(X_train, y_train)

# Prediction
y_pred = RFC.predict(X_test)

In [ ]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8379888268156425

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.86       105
           1       0.80      0.81      0.81        74

    accuracy                           0.84       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179

